In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import datetime

### UDF's

In [3]:
# Create Data audit Report for continuous variables
def continuous_var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  
                      x.std(), x.var(), x.min(), x.quantile(0.01), x.quantile(0.05),
                          x.quantile(0.10),x.quantile(0.25),x.quantile(0.50),x.quantile(0.75), 
                              x.quantile(0.90),x.quantile(0.95), x.quantile(0.99),x.max()], 
                  index = ['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1', 
                               'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

In [4]:
# Create Data audit Report for categorical variables
def categorical_var_summary(x):
    Mode = x.value_counts().sort_values(ascending = False)[0:1].reset_index()
    return pd.Series([x.count(), x.isnull().sum(), Mode.iloc[0, 0], Mode.iloc[0, 1], 
                          round(Mode.iloc[0, 1] * 100/x.count(), 2)], 
                  index = ['N', 'NMISS', 'MODE', 'FREQ', 'PERCENT'])

In [5]:
# Missing value imputation for categorical and continuous variables
def missing_imputation(x, stats = 'mean'):
    if (x.dtypes == 'float64') | (x.dtypes == 'int64'):
        x = x.fillna(x.mean()) if stats == 'mean' else x.fillna(x.median())
    return x

In [6]:
# An utility function to create dummy variable
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix = colname, drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

### data import and inspection

In [7]:
Attack_Back=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back.csv')
Attack_Back_BackButterOverflow=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_BufferOverflow.csv')
Attack_Back_FTPWrite=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_FTPWrite.csv')
Attack_Back_GuessPassword=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_GuessPassword.csv')
Attack_Back_Neptune=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_Neptune.csv')
Attack_Back_NMap=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_NMap.csv')
Attack_Back_Normal=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_Normal.csv')
Attack_Back_PortSweep=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_PortSweep.csv')
Attack_Back_RootKit=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_RootKit.csv')
Attack_Back_Satan=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_Satan.csv')
Attack_Back_Smurf=pd.read_csv('C:/Users/avnis/Case study/Machine Learning/CS 12/Data_of_Attack_Back_Smurf.csv')

In [8]:
# adding the new column in each data set as ATTACK with different set of attacks
Attack_Back['ATTACK']='Back'
Attack_Back_BackButterOverflow['ATTACK']='BackButterOverflow'
Attack_Back_FTPWrite['ATTACK']='FTPWrite'
Attack_Back_GuessPassword['ATTACK']='GuessPassword'
Attack_Back_Neptune['ATTACK']='Neptune'
Attack_Back_NMap['ATTACK']='NMap'
Attack_Back_PortSweep['ATTACK']='PortSweep'
Attack_Back_RootKit['ATTACK']='RootKit'
Attack_Back_Satan['ATTACK']='Satan'
Attack_Back_Smurf['ATTACK']='Smurf'
Attack_Back_Normal['ATTACK']='Normal'

In [9]:
# Concating each data set
Data=pd.concat([Attack_Back,Attack_Back_BackButterOverflow,Attack_Back_FTPWrite,
               Attack_Back_GuessPassword,Attack_Back_Neptune,Attack_Back_NMap,
               Attack_Back_Normal,Attack_Back_PortSweep,Attack_Back_RootKit,
               Attack_Back_Satan,Attack_Back_Smurf],axis=0,ignore_index=True)

In [10]:
Data.drop(['0.0026', ' 0', ' 0.07', ' 0.1',
       ' 0.00116',' 0.00451', ' 0.2', ' 0.3', ' 0.4', ' 0.2.1', ' 0.5',
       ' 0.1.1', ' 0.6', ' 0.7', ' 0.8', ' 0.9', ' 0.1.2', ' 0.10', ' 0.1.3',
       ' 0.11', ' 0.12', ' 0.1.4', ' 0.001', ' 0.001.1', ' 0.13', ' 0.14',
       ' 0.15', ' 0.16', ' 0.1.5', ' 0.17', ' 0.18', ' 0.001.2', ' 0.001.3',
       ' 0.1.6', ' 0.19', ' 0.1.7', ' 0.20', ' 0.21', ' 0.22', ' 0.23',
       ' 0.24'],axis=1,inplace=True)

In [11]:
Data.shape

(817550, 42)

In [12]:
Data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,ATTACK
0,0.0,0.00,0.00,0.0,0.54540,0.08314,0.0,0.0,0.0,0.2,...,0.001,0.100,0.000,0.100,0.0,0.0,0.0,0.0,0.0,Back
1,0.0,0.00,0.00,0.0,0.54540,0.08314,0.0,0.0,0.0,0.2,...,0.002,0.100,0.000,0.050,0.0,0.0,0.0,0.0,0.0,Back
2,0.0,0.00,0.00,0.0,0.54540,0.08314,0.0,0.0,0.0,0.2,...,0.003,0.100,0.000,0.033,0.0,0.0,0.0,0.0,0.0,Back
3,0.0,0.00,0.00,0.0,0.54540,0.08314,0.0,0.0,0.0,0.2,...,0.004,0.100,0.000,0.025,0.0,0.0,0.0,0.0,0.0,Back
4,0.0,0.00,0.00,0.0,0.54540,0.08314,0.0,0.0,0.0,0.2,...,0.005,0.100,0.000,0.020,0.0,0.0,0.0,0.0,0.0,Back
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817545,0.0,0.02,0.09,0.0,0.01032,0.00000,0.0,0.0,0.0,0.0,...,0.251,0.098,0.001,0.098,0.0,0.0,0.0,0.0,0.0,Smurf
817546,0.0,0.02,0.09,0.0,0.01032,0.00000,0.0,0.0,0.0,0.0,...,0.252,0.099,0.001,0.099,0.0,0.0,0.0,0.0,0.0,Smurf
817547,0.0,0.02,0.09,0.0,0.01032,0.00000,0.0,0.0,0.0,0.0,...,0.253,0.099,0.001,0.099,0.0,0.0,0.0,0.0,0.0,Smurf
817548,0.0,0.02,0.09,0.0,0.01032,0.00000,0.0,0.0,0.0,0.0,...,0.254,0.100,0.001,0.100,0.0,0.0,0.0,0.0,0.0,Smurf


In [13]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817550 entries, 0 to 817549
Data columns (total 42 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   duration                      817543 non-null  float64
 1    protocol_type                817543 non-null  float64
 2    service                      817543 non-null  float64
 3    flag                         817543 non-null  float64
 4    src_bytes                    817543 non-null  float64
 5    dst_bytes                    817543 non-null  float64
 6    land                         817543 non-null  float64
 7    wrong_fragment               817543 non-null  float64
 8    urgent                       817543 non-null  float64
 9    hot                          817543 non-null  float64
 10   num_failed_logins            817543 non-null  float64
 11   logged_in                    817543 non-null  float64
 12   num_compromised              817543 non-nul

### EXPLORATORY ANALYSIS

In [14]:
# summaries of continuous variables
Data.select_dtypes(['float64']).apply(continuous_var_summary)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
N,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.0,8.175430e+05,8.175430e+05,817543.000000,...,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000,817543.000000
NMISS,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.0,7.000000e+00,7.000000e+00,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000
SUM,716.808800,465.680000,43254.220000,16481.500000,4056.166760,20325.602660,0.0,3.000000e-01,1.800000e+00,2007.300000,...,136423.290000,132158.013000,54607.321000,2763.120000,5025.482000,1375.513000,20328.601000,20317.676000,4392.049000,4315.787000
MEAN,0.000877,0.000570,0.052908,0.020160,0.004961,0.024862,0.0,3.669532e-07,2.201719e-06,0.002455,...,0.166870,0.161653,0.066794,0.003380,0.006147,0.001682,0.024865,0.024852,0.005372,0.005279
MEDIAN,0.000000,0.000000,0.000000,0.000000,0.002170,0.003670,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.255000,0.255000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
STD,0.026636,0.002596,0.096388,0.030960,0.033235,0.066634,0.0,3.317920e-04,7.662382e-04,0.041469,...,0.102057,0.111642,0.043421,0.008556,0.017458,0.003708,0.043062,0.043152,0.021920,0.022070
VAR,0.000709,0.000007,0.009291,0.000958,0.001105,0.004440,0.0,1.100860e-07,5.871210e-07,0.001720,...,0.010416,0.012464,0.001885,0.000073,0.000305,0.000014,0.001854,0.001862,0.000480,0.000487
MIN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.001000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,...,0.006000,0.003000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
Data.ATTACK.value_counts()/Data.shape[0]

Normal                0.705413
Neptune               0.277938
Satan                 0.006139
Smurf                 0.003678
PortSweep             0.003625
NMap                  0.001901
Back                  0.001184
GuessPassword         0.000065
BackButterOverflow    0.000037
RootKit               0.000012
FTPWrite              0.000009
Name: ATTACK, dtype: float64

### Feature Selection

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [17]:
# get the y and X varaibales
y_var='ATTACK'
x_var=Data.columns.difference(['ATTACK'])

In [18]:
Data[x_var]=Data[x_var].apply(lambda x:missing_imputation(x))

In [20]:
#running logistic regression model
logreg=LogisticRegression(max_iter=1000)

In [21]:
# RFE for feature selection
rfe=RFE(logreg,n_features_to_select=20).fit(Data[x_var],Data[y_var])

In [22]:
rfe.support_

array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True, False,  True, False, False, False, False,
        True, False, False, False, False, False, False, False,  True,
        True, False,  True,  True,  True, False, False,  True,  True,
        True, False, False, False, False])

In [23]:
X_final=Data[x_var[rfe.support_]]
Y_final=Data['ATTACK']

### Build the model

In [39]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [25]:
# data set partition
X_train,X_test,Y_train,Y_test=train_test_split(X_final,Y_final,
                                               train_size=0.7,
                                              random_state=1234)

### Using the Naive Bayse model

In [26]:
model_NB=BernoulliNB().fit(X_train,Y_train)

#### Evaluate Naive bayse model

In [27]:
#Accuracy score
accuracy=metrics.accuracy_score( Y_train, model_NB.predict( X_train ) )
print("GaussianNB model accuracy on train data: %.3f" % (accuracy))

GaussianNB model accuracy on train data: 0.926


### Validate The model

In [28]:
#Accuracy score
accuracy=metrics.accuracy_score( Y_test, model_NB.predict( X_test ) )
print("GaussianNB model accuracy on test data: %.3f" % (accuracy))

GaussianNB model accuracy on test data: 0.926


### Trying a SVM Model

In [29]:
#standardization of X_variable
from sklearn.preprocessing import StandardScaler
X_scaled = pd.DataFrame( StandardScaler().fit_transform( X_final ) )
X_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.639866,-0.290174,-1.625275,-0.395024,-0.24509,5.376059,0.764741,-0.577439,-1.439009,-0.239188,-0.651169,0.753069,-0.219448,-0.239825,0.634593,-0.576534,-0.548906,3.817625,-0.577689,2.011555
1,-0.628995,-0.290174,-1.615477,-0.395024,-0.24509,2.511973,0.764741,-0.577439,-1.430052,-0.239188,-0.651169,0.753069,-0.219448,-0.239825,0.634593,-0.576534,-0.548906,2.416829,-0.577689,1.246302
2,-0.618123,-0.290174,-1.605678,-0.395024,-0.24509,1.538183,0.764741,-0.577439,-1.421095,-0.239188,-0.651169,0.753069,-0.219448,-0.239825,0.634593,-0.576534,-0.548906,1.695208,-0.577689,0.886183
3,-0.607252,-0.290174,-1.595880,-0.395024,-0.24509,1.079929,0.764741,-0.577439,-1.412137,-0.239188,-0.651169,0.753069,-0.219448,-0.239825,0.634593,-0.576534,-0.548906,-0.427209,-0.577689,-0.239188
4,-0.607252,-0.290174,-1.586081,-0.395024,-0.24509,0.793521,0.764741,-0.577439,-1.403180,-0.239188,-0.651169,0.753069,-0.219448,-0.239825,0.634593,-0.576534,-0.548906,-0.427209,-0.577689,-0.239188


In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_scaled, 
                                                    Y_final, 
                                                    test_size = 0.3, 
                                                    random_state = 1234)

### Using SVM

In [31]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics

In [33]:
# initialize the model
m_SVC = SVC( class_weight = 'balanced', gamma = 'auto' )

In [34]:
# tuning parameters
params = {'C': [5, 10, 15], 
              'kernel': ['linear']}

# grid search and cv
model_svm = GridSearchCV( estimator = m_SVC, 
                         param_grid = params, 
                         cv = 3, 
                         verbose = 10,
                         scoring = 'accuracy').fit( x_train, y_train )

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3; 1/3] START C=5, kernel=linear..........................................
[CV 1/3; 1/3] END ........................C=5, kernel=linear; total time=12.6min
[CV 2/3; 1/3] START C=5, kernel=linear..........................................
[CV 2/3; 1/3] END ........................C=5, kernel=linear; total time=10.7min
[CV 3/3; 1/3] START C=5, kernel=linear..........................................
[CV 3/3; 1/3] END ........................C=5, kernel=linear; total time=11.4min
[CV 1/3; 2/3] START C=10, kernel=linear.........................................
[CV 1/3; 2/3] END .......................C=10, kernel=linear; total time=11.7min
[CV 2/3; 2/3] START C=10, kernel=linear.........................................
[CV 2/3; 2/3] END .......................C=10, kernel=linear; total time=10.0min
[CV 3/3; 2/3] START C=10, kernel=linear.........................................
[CV 3/3; 2/3] END .......................C=10, ker

In [36]:
model_svm.best_score_

0.9899752733694444

### Evaluate the  SVM model

In [37]:
#Accuracy score
accuracy=metrics.accuracy_score( y_train, model_NB.predict( x_train ) )
print("SVM model accuracy on train data: %.3f" % (accuracy))

SVM model accuracy on train data: 0.531


### Validate The model

In [38]:
#Accuracy score
accuracy=metrics.accuracy_score( y_test, model_NB.predict( x_test ) )
print("SVM model Validation accuracy on test data: %.3f" % (accuracy))

SVM model Validation accuracy on test data: 0.532
